In [14]:
# Lessons Learned 
# =========================================
# 1/ when getting errors, do Cell->Run All , so they may go away
# 2/ It seems when using Def functions instead of Flat code with out Defs , we are getting errors
# 3/ But Def will help to use local variable names with out global scope , can use same var names  df etc...

In [69]:
#import desired packages
import numpy as np
import pandas as pd
import traceback
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

In [70]:
import requests
def extract_target_price(ticker):
    url = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/"
    url = url + ticker + "?formatted=true&crumb=RqWcRwtueFe&lang=en-US&region=US&modules=upgradeDowngradeHistory%2CrecommendationTrend%2CfinancialData%2CearningsHistory%2CearningsTrend%2CindustryTrend&corsDomain=finance.yahoo.com"
    #print(url)
    try:
        resp = requests.get(url)
        return resp.json()
    except: 
        traceback.print_exc()
        pprint("Requests ERROR for ticker:" + ticker)
        return False
    

In [71]:
import time

def get_target_price_record(ticker): 
    json_data = extract_target_price(ticker)
    #pprint(resp)
    s2 = json_data['quoteSummary']['result'][0] # there is a List( []), so get 0 th element
    s = s2['financialData']

    # Analyst Traget Prices
    current = float(s['currentPrice']['fmt'])
    low = float(s['targetLowPrice']['fmt'])
    median = float(s['targetMedianPrice']['fmt'])
    mean = float(s['targetMeanPrice']['fmt'])
    hi = float(s['targetHighPrice']['fmt'])

    today = time.strftime("%Y-%m-%d")
    target_price_rec = [ticker, today, current, low, median, mean, hi]
    return target_price_rec


In [72]:
def create_tatprice_lstrget_price_masterfile():

    #import SnP Symbol list 
    snp_tickers_allyears = pd.read_csv('data-SnP-10years-Tickers-byyear-dfready.csv', header=0)

    tprice_lst = []
    nodata_found_lst = []

    year = 2015 
    #llst = snp_tickers_allyears.query('Year == @year').values.tolist()
    llst = [ ['Ticker', 'Year'], ['FIT', 2008]]  # use this for Testing one ROW
    
    for stock in llst[1:] :  # skip 0 th element being it header
        # here index are : 0 - TIcker , 1 - year
        #print('trying to fetch data for ' + stock[0])
        try:
            rec = get_target_price_record(stock[0])  
            if len(rec) > 0:
                tprice_lst.append(rec)
                print( rec + ' good')
            else:
                print(stock[0])
                nodata_found_lst.append(stock)
        except:
            nodata_found_lst.append(stock)
            #print('Exception for { }, year {}'.format(stock[0], stock[1])

    df = pd.DataFrame(tprice_lst)
    cols = ['Ticker', 'Date', 'Current', 'Low_tp', 'Median_tp', 'Mean_tp', 'High_tp']
    df.columns = cols
    df['Below_median'] = round( ((df['Current'] - df['Median_tp']) * 100 )/df['Current'], 1) 
    df['Below_mean'] = round( ((df['Current'] - df['Mean_tp']) * 100 )/df['Current'], 1)
    print(cols)
    df2 = df.sort_values(by='Below_median', ascending=True)
    df2.to_csv('data-analyst-target-price.csv', index=False)

    print(df2)
    nodata_df = pd.DataFrame(nodata_found_lst)
    nodata_df.columns = ['Ticker', 'Year']
    nodata_df.to_csv('data-nodata-found-target-price.csv', index=False)


In [73]:
create_target_price_masterfile()

FIT is good
